In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding,CuDNNGRU
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [ ]:
train_dataset=pd.read_csv('../input/twitter-sentiment-analysis-hatred-speech/train.csv')
test_dataset=pd.read_csv('../input/twitter-sentiment-analysis-hatred-speech/test.csv')

In [ ]:
#set index
train_dataset.set_index('id',inplace=True)
test_dataset.set_index('id',inplace=True)

In [ ]:
#to list
target=train_dataset['label'].values.tolist()
data=train_dataset['tweet'].values.tolist()

#splitting data into train and test
cutoff=int(len(data)*0.8)
x_train, x_test=data[:cutoff],data[cutoff:]
y_train, y_test=target[:cutoff],target[cutoff:]

y_train=np.array(y_train)
y_test=np.array(y_test)

In [ ]:
#creating a threshold for number of words 
num_words=10000 

#creating a tokenizer
tokenizer=Tokenizer(num_words=num_words)

#converting words to numbers
tokenizer.fit_on_texts(data) 

#creating vectors
x_train_tokens=tokenizer.texts_to_sequences(x_train)
x_test_tokens=tokenizer.texts_to_sequences(x_test)


In [ ]:
print(x_train[800])
print(x_train_tokens[800])

In [ ]:
num_tokens=[len(tokens) for tokens in x_train_tokens+x_test_tokens]
num_tokens=np.array(num_tokens)
max_tokens=np.mean(num_tokens)+2*np.std(num_tokens)
max_tokens=int(max_tokens)

In [ ]:
np.sum(num_tokens<max_tokens)/len(num_tokens)

In [ ]:
#padding operation
x_train_pad=pad_sequences(x_train_tokens,
                              maxlen=max_tokens)
x_test_pad=pad_sequences(x_test_tokens,
                         maxlen=max_tokens)

In [ ]:
#creating model
model=Sequential()
embedding_size=50  # we will create a 50 size vector for each word.

model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer')
) # this Embedding layer will take a text as an input, convert it to a vector as an output

model.add(GRU(units=16, 
              return_sequences=True) 
)
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1,activation='sigmoid'))
optimizer=Adam(lr=1e-3)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(x_train_pad,
          y_train,
          epochs=5,
          batch_size=256)

In [ ]:
result=model.evaluate(x_test_pad,
                      y_test)

In [ ]:
#we will go over the results and false predictions and try to understand this errors.
y_pred=model.predict(x=x_test_pad[0:1000])
y_pred=y_pred.T[0]

cls_pred=np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

cls_true=np.array(y_test[0:1000])

incorrect=np.where(cls_pred!=cls_true) 
incorrect=incorrect[0] 

print(len(incorrect)) 

In [ ]:
idx=incorrect[2] # the first false prediction index
text=x_test[idx]
print(text)
print(y_pred[idx]) # the probability
print(cls_true[idx])

In [ ]:
#scoring test dataset
test_data=test_dataset['tweet'].values.tolist()

test_data_tokens=tokenizer.texts_to_sequences(test_data)

test_data_tokens_pad=pad_sequences(test_data_tokens,
                         maxlen=max_tokens)

test_data_predictions=model.predict(test_data_tokens_pad)
test_data_predictions=[1 if prob>0.5 else 0 for prob in test_data_predictions]
test_dataset['label']=test_data_predictions

In [ ]:
test_dataset.to_csv('prediction.csv',index=False)